In [2]:
%pip install optuna

Note: you may need to restart the kernel to use updated packages.


In [3]:
# Forma directa de obtener el dataset de heart disease
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.metrics import r2_score
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, KFold
from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_auc_score
from sklearn.multiclass import OneVsRestClassifier
import warnings
warnings.filterwarnings('ignore')
import optuna


In [4]:
df=pd.read_csv(r"C:\Users\marce\OneDrive - ITESO\Quinto semestre\Lab estadistico\brain_tumor_dataset.csv")

In [5]:
df.head()

,Patient_ID,Age,Gender,Tumor_Type,Tumor_Size,Location,Histology,Stage,Symptom_1,Symptom_2,Symptom_3,Radiation_Treatment,Surgery_Performed,Chemotherapy,Survival_Rate,Tumor_Growth_Rate,Family_History,MRI_Result,Follow_Up_Required
0,1,73,Male,Malignant,5.375612,Temporal,Astrocytoma,III,Vision Issues,Seizures,Seizures,No,No,No,51.312579,0.111876,No,Positive,Yes
1,2,26,Male,Benign,4.847098,Parietal,Glioblastoma,II,Headache,Headache,Nausea,Yes,Yes,Yes,46.373273,2.165736,Yes,Positive,Yes
2,3,31,Male,Benign,5.588391,Parietal,Meningioma,I,Vision Issues,Headache,Seizures,No,No,No,47.072221,1.884228,No,Negative,No
3,4,29,Male,Malignant,1.436600,Temporal,Medulloblastoma,IV,Vision Issues,Seizures,Headache,Yes,No,Yes,51.853634,1.283342,Yes,Negative,No
4,5,54,Female,Benign,2.417506,Parietal,Glioblastoma,I,Headache,Headache,Seizures,No,No,Yes,54.708987,2.069477,No,Positive,Yes


In [6]:
df=df.dropna()
df.describe()

,Patient_ID,Age,Tumor_Size,Survival_Rate,Tumor_Growth_Rate
count,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000
mean,10000.500000,49.433700,5.236552,70.131699,1.547701
std,5773.647028,17.389296,2.754359,17.270834,0.835995
min,1.000000,20.000000,0.500410,40.002386,0.100017
25%,5000.750000,34.000000,2.836686,55.224439,0.830433
50%,10000.500000,49.000000,5.200675,70.305066,1.541821
75%,15000.250000,65.000000,7.624299,85.043395,2.262484
max,20000.000000,79.000000,9.999420,99.998125,2.999835


In [7]:
df=df.drop_duplicates()
df.describe()
df=df.drop(columns='Patient_ID')

In [8]:
dfdummies=pd.get_dummies(df,columns=["Symptom_1","Symptom_2","Symptom_3","Stage","Location","Histology"])

cols=dfdummies.columns
cols=cols.drop('Tumor_Type')
dfdummies=dfdummies.replace(["No","Yes","Positive","Negative","Male","Female",False,True,"Malignant","Benign"],[0,1,1,0,1,0,0,1,0,1])
dfdummies.columns
dfdummies.head()


,Age,Gender,Tumor_Type,Tumor_Size,Radiation_Treatment,Surgery_Performed,Chemotherapy,Survival_Rate,Tumor_Growth_Rate,Family_History,...,Stage_III,Stage_IV,Location_Frontal,Location_Occipital,Location_Parietal,Location_Temporal,Histology_Astrocytoma,Histology_Glioblastoma,Histology_Medulloblastoma,Histology_Meningioma
0,73,1,0,5.375612,0,0,0,51.312579,0.111876,0,...,1,0,0,0,0,1,1,0,0,0
1,26,1,1,4.847098,1,1,1,46.373273,2.165736,1,...,0,0,0,0,1,0,0,1,0,0
2,31,1,1,5.588391,0,0,0,47.072221,1.884228,0,...,0,0,0,0,1,0,0,0,0,1
3,29,1,0,1.436600,1,0,1,51.853634,1.283342,1,...,0,1,0,0,0,1,0,0,1,0
4,54,0,1,2.417506,0,0,1,54.708987,2.069477,0,...,0,0,0,0,1,0,0,1,0,0


In [9]:
x=dfdummies[cols]
y=dfdummies['Tumor_Type']
modelo=sm.OLS(y,x)
results=modelo.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             Tumor_Type   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                 -0.000
Method:                 Least Squares   F-statistic:                    0.9894
Date:                Wed, 29 Oct 2025   Prob (F-statistic):              0.481
Time:                        17:56:35   Log-Likelihood:                -14501.
No. Observations:               20000   AIC:                         2.906e+04
Df Residuals:                   19970   BIC:                         2.930e+04
Df Model:                          29                                         
Covariance Type:            nonrobust                                         
=============================================================================================
                                coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------
Age                          -0.0002      0.000     -0.843      0.399      -0.001       0.000
Gender                        0.0038      0.007      0.540      0.589      -0.010       0.018
Tumor_Size                    0.0025      0.001      1.913      0.056   -6.04e-05       0.005
Radiation_Treatment          -0.0097      0.007     -1.364      0.173      -0.024       0.004
Surgery_Performed             0.0036      0.007      0.510      0.610      -0.010       0.017
Chemotherapy                 -0.0047      0.007     -0.659      0.510      -0.019       0.009
Survival_Rate              7.911e-05      0.000      0.386      0.699      -0.000       0.000
Tumor_Growth_Rate             0.0004      0.004      0.101      0.919      -0.008       0.009
Family_History            -4.898e-05      0.007     -0.007      0.994      -0.014       0.014
MRI_Result                   -0.0011      0.007     -0.152      0.879      -0.015       0.013
Follow_Up_Required           -0.0142      0.007     -2.011      0.044      -0.028      -0.000
Symptom_1_Headache            0.0851      0.007     11.895      0.000       0.071       0.099
Symptom_1_Nausea              0.0789      0.007     10.991      0.000       0.065       0.093
Symptom_1_Seizures            0.0886      0.007     12.326      0.000       0.075       0.103
Symptom_1_Vision Issues       0.0802      0.007     11.161      0.000       0.066       0.094
Symptom_2_Headache            0.0867      0.007     12.035      0.000       0.073       0.101
Symptom_2_Nausea              0.0907      0.007     12.572      0.000       0.077       0.105
Symptom_2_Seizures            0.0850      0.007     11.912      0.000       0.071       0.099
Symptom_2_Vision Issues       0.0704      0.007      9.843      0.000       0.056       0.084
Symptom_3_Headache            0.0845      0.007     11.754      0.000       0.070       0.099
Symptom_3_Nausea              0.0865      0.007     12.049      0.000       0.072       0.101
Symptom_3_Seizures            0.0862      0.007     12.052      0.000       0.072       0.100
Symptom_3_Vision Issues       0.0756      0.007     10.517      0.000       0.062       0.090
Stage_I                       0.0749      0.007     10.493      0.000       0.061       0.089
Stage_II                      0.0877      0.007     12.240      0.000       0.074       0.102
Stage_III                     0.0775      0.007     10.744      0.000       0.063       0.092
Stage_IV                      0.0926      0.007     12.891      0.000       0.079       0.107
Location_Frontal              0.0875      0.007     12.082      0.000       0.073       0.102
Location_Occipital            0.0918      0.007     12.772      0.000       0.078       0.106
Location_Parietal             0.0822      0.007     11.469      0.000      

In [10]:
cols=cols.drop(['MRI_Result','Family_History','Tumor_Growth_Rate','Age','Gender','Radiation_Treatment','Surgery_Performed','Chemotherapy','Survival_Rate'])


In [11]:
x=dfdummies[cols]
y=dfdummies['Tumor_Type']
LOGR = LogisticRegression()
LOGR.fit(x, y)
LOGR.score(x,y)

0.5122

In [12]:
dfshort=dfdummies.head(200)
x=dfshort[cols]
y=dfshort['Tumor_Type']

In [13]:
kfold = KFold(n_splits=10, shuffle=True, random_state=42)
svc_model = SVC(kernel='linear', probability=True, random_state=42)
auc_scores = cross_val_score(svc_model, x, y, cv=kfold, scoring='roc_auc_ovr')
print(np.mean(auc_scores), np.std(auc_scores))



0.519229575979576 0.08591609986920777


In [14]:
def objective(trial):
    C_value = trial.suggest_loguniform('C', .00001, 100)
    
    class_weight_choice = trial.suggest_categorical('class_weight', ['balanced', None])
    classifier = SVC(
        kernel='linear',
        C=C_value,
        class_weight=class_weight_choice,
        random_state=42
    )

    score = cross_val_score(classifier, x, y, n_jobs=-1, cv=10, scoring='accuracy')
    return score.mean()

In [15]:
study=optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100)
print("Optimización Completa.")

print(f"Valor (Precisión media): {study.best_value:.4f}")
print(f"Mejores Hiperparámetros:")

# Muestra los mejores parámetros encontrados
for key, value in study.best_params.items():
    print(f"    {key}: {value}")

[I 2025-10-29 17:56:35,403] A new study created in memory with name: no-name-c431f9ce-e2bb-40ba-aa4c-b2a2717f1747
[I 2025-10-29 17:56:38,128] Trial 0 finished with value: 0.515 and parameters: {'C': 0.0011610948472862404, 'class_weight': None}. Best is trial 0 with value: 0.515.
[I 2025-10-29 17:56:39,198] Trial 1 finished with value: 0.44000000000000006 and parameters: {'C': 1.6053960200647373, 'class_weight': 'balanced'}. Best is trial 0 with value: 0.515.
[I 2025-10-29 17:56:39,224] Trial 2 finished with value: 0.515 and parameters: {'C': 0.00040478764867691685, 'class_weight': None}. Best is trial 0 with value: 0.515.
[I 2025-10-29 17:56:39,250] Trial 3 finished with value: 0.515 and parameters: {'C': 6.286162706541178e-05, 'class_weight': 'balanced'}. Best is trial 0 with value: 0.515.
[I 2025-10-29 17:56:39,277] Trial 4 finished with value: 0.515 and parameters: {'C': 1.921858684654751e-05, 'class_weight': 'balanced'}. Best is trial 0 with value: 0.515.
[I 2025-10-29 17:56:39,302

Optimización Completa.
Valor (Precisión media): 0.5850
Mejores Hiperparámetros:
    C: 0.0023534402046864354
    class_weight: None


In [16]:
x=dfdummies[cols]
y=dfdummies['Tumor_Type']

def objective(trial):
    C_value = trial.suggest_loguniform('C', .00001, 100)
    
    class_weight_choice = trial.suggest_categorical('class_weight', ['balanced', None])
    classifier = SVC(
        kernel='linear',
        C=C_value,
        class_weight=class_weight_choice,
        random_state=42
    )

    score = cross_val_score(classifier, x, y, n_jobs=-1, cv=10, scoring='accuracy')
    return score.mean()


In [17]:
study=optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100)
print("Optimización Completa.")

print(f"Valor (Precisión media): {study.best_value:.4f}")
print(f"Mejores Hiperparámetros:")

# Muestra los mejores parámetros encontrados
for key, value in study.best_params.items():
    print(f"    {key}: {value}")

[I 2025-10-29 17:56:42,489] A new study created in memory with name: no-name-61b0698f-c0b9-4a05-a789-60c3a3b43acb
[I 2025-10-29 17:57:42,333] Trial 0 finished with value: 0.5057999999999999 and parameters: {'C': 0.006002907815715446, 'class_weight': 'balanced'}. Best is trial 0 with value: 0.5057999999999999.
[I 2025-10-29 17:58:29,249] Trial 1 finished with value: 0.5049499999999999 and parameters: {'C': 0.00042614610235233074, 'class_weight': None}. Best is trial 0 with value: 0.5057999999999999.
[I 2025-10-29 17:59:28,237] Trial 2 finished with value: 0.50495 and parameters: {'C': 0.004415939810998749, 'class_weight': 'balanced'}. Best is trial 0 with value: 0.5057999999999999.
[I 2025-10-29 18:00:14,775] Trial 3 finished with value: 0.5003 and parameters: {'C': 0.0003286557624410055, 'class_weight': None}. Best is trial 0 with value: 0.5057999999999999.
[I 2025-10-29 18:01:01,471] Trial 4 finished with value: 0.5074 and parameters: {'C': 0.001098567054770132, 'class_weight': None}.

Optimización Completa.
Valor (Precisión media): 0.5088
Mejores Hiperparámetros:
    C: 0.0007299820453571904
    class_weight: balanced
